In [3]:
HIERARCHY_MAX_LEN = 7

In [4]:
import torch.nn as nn
import torch
import swifter
import pandas as pd

In [5]:
def add_product_to_hier(row):
    temp_hier = row['hierarchy']
    temp_hier.extend([row['Key_product']])
    row['hierarchy'] = temp_hier
    return row

df = pd.read_pickle("../raw_data/lines_hier.pkl")
df = df.swifter.apply(add_product_to_hier, axis=1)
df['hierarchy'] = df['hierarchy'].map(lambda x: ([0]*HIERARCHY_MAX_LEN + x)[-HIERARCHY_MAX_LEN:])
df.head(2)

Pandas Apply:   0%|          | 0/1814677 [00:00<?, ?it/s]